Modelo de recomendaciones ML

In [326]:
import pandas as pd
import gzip 
import numpy as np
import pickle

In [327]:
steam_games = pd.read_csv('../datasets/steam_games.csv')
user_items = pd.read_csv('../datasets/user_items.csv.gz')

In [328]:
steam_games

,publisher,genres,app_name,title,release_date,price,id,year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,2018.0
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,2018-01-04,0.00,643980,2018.0
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,0.00,670290,2017.0
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,2017-12-07,0.99,767400,2017.0
4,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,3.99,772540,2018.0
...,...,...,...,...,...,...,...,...
27552,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,Kebab it Up!,2018-01-04,1.99,745400,2018.0
27553,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,2018-01-04,1.99,773640,2018.0
27554,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,4.99,733530,2018.0
27555,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,2018-01-04,1.99,610660,2018.0


Seleccionamos las columnas que vamos a usar para nuestra matriz de caracteristicas

In [329]:
df_steam_games = steam_games[['id', 'genres']]

In [330]:
# Desglosar las listas en la columna 'genres'
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: eval(x))

C:\Users\Willc\AppData\Local\Temp\ipykernel_2904\1070283543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: eval(x))


Generamos las columnas dummies apartir de la columna genres

In [331]:
from sklearn.preprocessing import MultiLabelBinarizer  
# Crear una instancia de MultiLabelBinarizer y ajustarla a los géneros
mlb = MultiLabelBinarizer()
dummies = pd.DataFrame(mlb.fit_transform(df_steam_games['genres']), columns=mlb.classes_, index=df_steam_games.index)

# Combinar las dummies con el DataFrame original y eliminar duplicados en la columna title
df_steam_games_with_dummies = pd.concat([df_steam_games.drop('genres', axis=1), dummies], axis=1)
df_steam_games_with_dummies = df_steam_games_with_dummies.drop_duplicates(subset='id')


In [332]:
df_steam_games_with_dummies

,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,643980,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,670290,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,767400,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27552,745400,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27553,773640,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
27554,733530,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
27555,610660,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [333]:
df_steam_games_with_dummies.set_index('id', inplace=True)

In [334]:
steam_game_id_title = steam_games[['id', 'title']]

In [335]:
steam_game_id_title.head()

,id,title
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer


Ahora exportamos los df como csv

In [336]:
with gzip.open('../datasets/df_steam_games_with_dummies.csv.gz', 'wb') as f:
    df_steam_games_with_dummies.to_csv(f, encoding='utf_8', index=True)

In [337]:
df_steam_games_with_dummies

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
id,,,,,,,,,,,,,,,,,,,,,
761140,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
643980,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
670290,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
767400,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
772540,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745400,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
773640,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
733530,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [393]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_juego(id_producto:int):
    '''Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.'''

    if not isinstance(id_producto, int):
        try:
            id_producto = int(id_producto)
        except ValueError:
            return 'El Id debe ser un número entero'

    df_steam_games_with_dummies = pd.read_csv('../datasets/df_steam_games_with_dummies.csv.gz')
    steam_game_id_title = pd.read_csv('../datasets/steam_games_id_title.csv.gz')
    #Verificamos si el id ingresado esta en la base de datos
    if id_producto not in df_steam_games_with_dummies['id'].unique():
        return "ID no encontrado"
    #convertimos la columna id en index
    df_steam_games_with_dummies.set_index('id', inplace=True)

    # Obtener las características del juego dado su ID
    juego_caracteristicas = df_steam_games_with_dummies.loc[id_producto].values.reshape(1, -1)

    # Calcular la similitud del coseno entre el juego dado y todos los otros juegos
    similarities = cosine_similarity(df_steam_games_with_dummies.values, juego_caracteristicas)

    # Ordenar los juegos según su similitud y tomar los 6 juegos más similares (el primero es el mismo juego)
    similar_juegos_indices = similarities.flatten().argsort()[-6:-1][::-1]

    # Obtener los títulos de los juegos recomendados
    recommended_juegos = steam_game_id_title.loc[similar_juegos_indices, 'title'].tolist()

    return recommended_juegos

In [382]:
a = 1

In [384]:
type(a)

int

In [394]:
recomendaciones = recomendacion_juego(761140)
print(recomendaciones)

['FSX: Steam Edition - Around The World In 80 Flights Add-On', "Zombidle - Burrito's Pack", 'Football Manager Touch 2018 - No Loan Restrictions', 'Football Manager Touch 2018', 'Crashed Lander Demo']


Segundo modelo

In [340]:
user_reviews = pd.read_csv('../datasets/user_review.csv',usecols=['user_id','item_id','sentiment_analysis','recommend'])

In [341]:
user_reviews['user_id'].value_counts()

user_id
76561198108415635      10
NanoPi                 10
banksyyo               10
snubbo                 10
BuffinMutton           10
                       ..
554076033               1
_maximus                1
maxstupo                1
maxy21                  1
SkullainnLovesGoats     1
Name: count, Length: 25458, dtype: int64

In [342]:
steam_games.head()

,publisher,genres,app_name,title,release_date,price,id,year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,2018.0
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,2018-01-04,0.00,643980,2018.0
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,0.00,670290,2017.0
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,2017-12-07,0.99,767400,2017.0
4,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,3.99,772540,2018.0


In [343]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#elimino la columna user_id
user_reviews['user_id_num'] = label_encoder.fit_transform(user_reviews['user_id'])

In [344]:
user_reviews['rating'] = np.where(user_reviews['recommend'] == True,  # Si 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 5,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 3,  # Si 'sentimiento' es neutro
                                                   1)),  # Si 'sentimiento' es negativo cuando 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 4,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 2,  # Si 'sentimiento' es neutro
                                                   0)))  # Si 'sentimiento' es negativo cuando 'recommend' es False

In [345]:
user_reviews.to_csv('../datasets/user_review_rating.csv.gz',index=False)

In [346]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Suponiendo que tienes un DataFrame llamado 'df_ratings' con las columnas 'user_id', 'item_id' y 'rating'
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_reviews[['user_id', 'item_id', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

In [347]:
# Importar GridSearchCV
from surprise.model_selection import GridSearchCV

# Definir los parámetros para la búsqueda de cuadrícula
param_grid = {'n_factors': [5, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.002, 0.005], 'reg_all': [0.002, 0.02, 0.2]}

# Crear un objeto GridSearchCV con SVD como modelo, RMSE como métrica a optimizar, validación cruzada de 3 pliegues y uso de todos los núcleos disponibles
gs = GridSearchCV(algo_class=SVD, param_grid=param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# Ejecutar la búsqueda de cuadrícula en los datos
gs.fit(data)

# Obtener los mejores resultados de la búsqueda
best_rmse = gs.best_score['rmse']
best_params = gs.best_params['rmse']

print(f"Mejor RMSE: {best_rmse}")
print(f"Mejores parámetros: {best_params}")

Mejor RMSE: 1.5889592032467812
Mejores parámetros: {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


In [348]:
# Crear una instancia del modelo SVD y entrenarlo con el conjunto de entrenamiento
model = SVD(n_factors = 5, n_epochs = 20, lr_all = 0.005, reg_all = 0.2)
model.fit(trainset)

# Hacer predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular la precisión de las predicciones
accuracy.rmse(predictions)


RMSE: 1.5904


1.5903731808086947

In [349]:
with open('../datasets/SVD_model.pkl', 'wb') as file: # Exporto mi modelo
    pickle.dump(model, file)

In [374]:
def recomendacion_usuario(id_usuario:str):
    ''' Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.'''
    #importamos los datasets
    df_ratings = pd.read_csv('../datasets/user_review_rating.csv.gz')
    steam_game_id_title = pd.read_csv('../datasets/steam_games_id_title.csv.gz')

    with open('../datasets/SVD_model.pkl', 'rb') as archivo:
        model = pickle.load(archivo)

    #Verificamos si el usuario esta en la base de datos
    if id_usuario not in df_ratings['user_id'].unique():
        return "ID no encontrado"
    
    # Obtener todos los juegos disponibles
    todos_los_juegos = df_ratings['item_id'].unique()

    # Obtener los juegos valorados por el usuario
    juegos_valorados_por_usuario = df_ratings[df_ratings['user_id'] == id_usuario]['item_id'].unique()

    # Obtener los juegos no valorados por el usuario
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados_por_usuario))

    # Crear un DataFrame con los juegos no valorados por el usuario
    df_juegos_no_valorados = pd.DataFrame(juegos_no_valorados, columns=['item_id'])

    # Hacer predicciones para los juegos no valorados por el usuario
    df_juegos_no_valorados['prediccion'] = df_juegos_no_valorados['item_id'].apply(lambda x: model.predict(id_usuario, x).est)

    # Ordenar los juegos por la calificación predicha y tomar los primeros n juegos como recomendación
    juegos_recomendados = df_juegos_no_valorados.sort_values(by='prediccion', ascending=False)['item_id'].tolist()

    # Obtener los títulos de los juegos recomendados que existen en steam_game_id_title
    recommended_juegos = []
    for juego_id in juegos_recomendados:
       juego_titulo = steam_game_id_title.loc[steam_game_id_title['id'] == juego_id, 'title'].tolist()
       if juego_titulo:
            recommended_juegos.append(juego_titulo[0])
            if len(recommended_juegos) == 5:
                break

    return recommended_juegos

In [361]:
recomendacion_usuario('NanoPi')

['FINAL FANTASY VII',
 'Go! Go! Nippon! ~My First Trip to Japan~',
 'Rogue Legacy',
 'FEZ',
 'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™']